In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import importlib
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from flearn.utils.model_utils import read_data
import os
import importlib
import random
import numpy as np

from flearn.models import client
from flearn.models import group
from flearn.trainers import grouprox
from flearn.trainers import fedprox
from flearn.trainers import fedavg

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
params = {}
#DATASETS = ['sent140', 'nist', 'shakespeare', 'mnist', 'synthetic_iid', 'synthetic_0_0', 'synthetic_0.5_0.5', 'synthetic_1_1']

"""
Parameters that need to be manually set
"""

#params['dataset'] = 'fmnist'
params['dataset'] = 'nist'
#params['model'] = 'mlp'
params['model'] = 'mclr'
params['optimizer'] = 'grouprox'
params['drop_percent'] = 0

if params['dataset'].startswith('mnist'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.03
  params['mu'] = 1
  params['num_group'] = 3
if params['dataset'] == 'nist':
  params['model_params'] = (26,)
  params['learning_rate'] = 0.003
  params['mu'] = 1
  params['num_group'] = 5
if params['dataset'] == 'sent140':
  params['model_params'] = (25, 2, 100) # seq_len, num_classes, num_hidden
  params['learning_rate'] = 0.3
  params['mu'] = 0.01
  params['num_group'] = 5
if params['dataset'].startswith('synthetic'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.01
  params['mu'] = 1
  params['num_group'] = 5
if params['dataset'] == 'celeba':
  params['model_params'] = (2,)
  params['learning_rate'] = 0.01
  params['mu'] = 1
  params['num_group'] = 3
if params['dataset'] == 'vehicle':
  params['model_params'] = (2,)
  params['learning_rate'] = 0.01
  params['mu'] = 1
  params['num_group'] = 2
if params['dataset'] == 'fmnist':
  params['model_params'] = (10,)
  params['learning_rate'] = 0.05
  params['mu'] = 1
  params['num_group'] = 3

if params['optimizer']  == 'fedavg':
  pass
if params['optimizer']  == 'fedprox':
  pass
if params['optimizer']  == 'grouprox':
  params['MADC'] = False                       # If false, EDC will be used
  params['proximal'] = False
  params['RAC'] = False
  params['RCC'] = False
  params['recluster_epoch'] = None
  params['client_temp'] = None
  params['allow_empty'] = True
  params['agg_lr'] = 0.0 ########## Group Aggregataion Rate
  
  # TEST Hyperparameters, Experimental Only
  params['evenly'] = False
  params['min_clients'] = 2
    
  # We implement IFCA and FeSEM base on grouprox
  params['ifca'] = False
  params['fesem'] = False
  
  """
  We immplement IFCA and FeSEM base on grouprox,
  Some configurations should be flush for above two CFL frameworks
  """
  if params['ifca'] == True or params['fesem'] == True:
    params['proximal'] = False ### Do not modify it
    params['RAC'] = False
    params['RCC'] = False
    params['allow_empty'] = True
    params['agg_lr'] = 0.0 # IFCA and FeSEM don't use this parameter
    
"""
Common parameters
"""
params['num_rounds'] = 300
params['num_epochs'] = 10
params['batch_size'] = 10
params['eval_every'] = 1
params['clients_per_round'] = 20 ################ Important ######################
params['seed'] = 233

""" Set the output CSV file name """
info = '{}-{}-{}'.format(params['optimizer'], params['dataset'], params['model'])
info2 = ''
suffix = '.csv'
if params['optimizer'] == 'fedavg':
  pass
if params['optimizer'] == 'fedprox':
  pass
if params['optimizer'] == 'grouprox':
  if params['ifca'] == False and params['fesem'] == False:
    if params['MADC'] == True:
      info2 = info2 + '-MADC'
    else:
      info2 = info2 + '-EDC'
    if params['recluster_epoch']:
        info2 = info2 + '-RE{:2d}'.format(params['recluster_epoch'])
    if params['client_temp']:
        info2 = info2 + '-TEMP{:2d}'.format(params['client_temp'])
        
  if params['ifca'] == True:
    info2 = info2 + '-IFCA'
  if params['fesem'] == True:
    info2 = info2 + '-FeSEM'
  
  info2 = info2 +  '-NumG{:1d}-Agglr{:.4}-Proximal{:.1}'.format(
    params['num_group'], params['agg_lr'], str(params['proximal']))
  
  if params['RCC'] == True:
    info2 = info2 + '-RCC.{:.1}'.format(str(params['RCC']))
  
  if params['RAC'] == True:
    info2 = info2 + '-RAC.{:.1}-Evenly{:.1}'.format(str(params['RAC']), str(params['evenly']))
  elif params['allow_empty'] == True:
    info2 = info2 + '-Empty{:.1}'.format(str(params['allow_empty']))
  elif params['allow_empty'] == False:
    info2 = info2 + '-Evenly{:.1}'.format(str(params['evenly']))

params['export_filename'] = info + info2 + suffix

print(params['export_filename'])

In [ ]:
"""
Some preparations
"""

# read data
train_path = os.path.join('data', params['dataset'], 'data', 'train')
test_path = os.path.join('data', params['dataset'], 'data', 'test')
dataset = read_data(train_path, test_path)

# Load model
if params['dataset'].startswith('synthetic'):  # all synthetic datasets use the same model
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'synthetic', params['model'])
elif params['dataset'].startswith('mnist'):
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'mnist', params['model'])
else:
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', params['dataset'], params['model'])
mod = importlib.import_module(model_path)
learner = getattr(mod, 'Model')

# Set seeds
params['sklearn_seed'] = params['seed']
random.seed(1 + params['seed'])
np.random.seed(12 + params['seed'])
tf.set_random_seed(123 + params['seed'])

# Set server
if params['optimizer'] == 'fedavg':
  server = fedavg.Server(params, learner, dataset)  
if params['optimizer'] == 'fedprox':
  server = fedprox.Server(params, learner, dataset)
if params['optimizer'] == 'grouprox':
  server = grouprox.Server(params, learner, dataset)

In [ ]:
"""
Test Ternary Similarity
"""
#x, y = server.test_ternary_cosine_similariy()

In [ ]:
"""
import matplotlib.pyplot as plt
plt.figure(figsize=(7,7))
plt.scatter(x[:50000], y[:50000], linewidths=0.01, alpha=0.2, norm=True)
plt.show()
"""

In [ ]:
server.train()

In [ ]:
"""
fname = 'sampling_interval.csv'
interval = 5
rounds = 200
with open(fname, 'w') as csv_f:
  csv_f.write(str(1)+'\n')
  for i in range(2, rounds+1):
    if i % interval == 0:
      csv_f.write(str(i))
    elif i == rounds:
      csv_f.write('200')
    csv_f.write('\n')
"""

In [ ]:
a = np.arange(32).reshape(2,4,4)
mask = np.zeros(shape=(4,4))
np.fill_diagonal(mask, 1)
#for i in range(a.shape[0]):
#    mask[i,:,i] =1
#x = np.ma.array([a, mask=[0] + [1,0]*4)
b = np.repeat(mask[np.newaxis,:,:], 2, axis=0)
for i in range(b.shape[0]):
    b[i,:,i]=1
    #b[i,i,:]=1
print(a)
x=np.ma.array(a,mask=b)
print(x)
print(np.sum(x,axis=-1))